# Winning the GHZ Game on Real IBM Quantum Computer<a name="top"></a>

*Lennart Schulze, Dr. Jan-Rainer Lahmann, October 2020*

The GHZ game is a serious game for quantum computing to show the quantum mechanical property of entanglement. Three players are asked for two different properties A and B of an object, an each property can be one of two conditions 1 or 0. When all are asked for property A, 1 should appear two times or zero times. When two players are asked for property B, 1 should appear three times ore one time. When the players don't know what the other two were asked, respectively, there is no strategy in classical logic that can lead to winning in all cases. Using Quantum Computing, however, there is. If you wish to find out more about this game, click [here](https://github.com/JanLahmann/Fun-with-Quantum/blob/master/GHZ-Game.ipynb).

The Qiskit built-in QASM Aer-simulator for running the Quantum circuit of the GHZ game is perfect, giving us the results that we expected from theory. In reality, however, the imperfection of real Quantum Computing hardware returns "noisy" output due to various sources of errors. Luckily, there is a variety of options available to mitigate these effects, some of which we will apply and inspect in this notebook.

In the following, we will have a closer look by:
1. Comparing IBM Quantum devices [[Go](#devices)]
2. Manually optimizing our circuit for a specific device [[Go](#manual)]
3. Using the Qiskit transpiler optimization [[Go](#transpiler)]
4. Running Measurement Error Mitigation [[Go](#mem)]
1. Combining the techniques [[Go](#all)]

Appendix [[Go](#appendix)]
* A: Solution for chapter 5. [[Go](#appendix-a)]
* B: Running this notebook on real Quantum Computers [[Go](#appendix-b)]

_____

## Setup
[[Top](#top)]

First, let's do the necessary imports:

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, transpile # circuit creation
from qiskit.visualization import plot_histogram # visualize results
from qiskit_aer import AerSimulator # simulator

import pandas as pd # handy tools for calculation

Next we'll define our global variables: 3 qubits are needed for our circuit (GHZ state), and we use a high number of shots (=times a circuit is executed per run on a device, out of which the relative result distribution is calculated) to get consistent results:

In [ ]:
# global vars
n = 3 # number of qubits
shots = 8000 # high number of shots per execution

For the remainder of this notebook, this will be our reference circuit for assessment. Note that the GHZ game uses 4 distinct circuits. These, however, only differ in their measurment base and order, but all have the same core - the GHZ state, which is the entanglement of 3 qubits (see part before the first barrier).

In [ ]:
# basic circuit
name = "GHZ yyx default" # ghz in measurement base Y,Y,X
ghz = QuantumCircuit(n,n, name=name) # create circuit
ghz.h(0) # h gate
ghz.cx(0,1) # cnot gate
ghz.cx(0,2) # cnot gate
ghz.barrier()
ghz.sdg([0,1]) # s dagger gate
ghz.h(range(n)) # h gates
ghz.barrier()
ghz.measure(range(n), range(n)) # measurements
ghz.draw("mpl") # draw cirucit

In order to evaluate the accuracy of the results of executing the GHZ state on different devices and with different mitigation techniques, we need a metric which we will define as state accuracy. It represents the percentage of counts resulted from the experiment that belong to one of the four target states of the GHZ game ('001', '010', '100', '111'). Reaching either of them equals winning they game, therefore our metric represents the fidelity with which we win the GHZ game on our quantum computer. We will use the state accuracy to measure the effectiveness of our different optimization approaches.

In [ ]:
expected_states = ['001', '010', '100', '111']  # target states

def get_state_accuracy(counts): # funciton to calculate state accuracy
    expected_counts = 0
    for state in counts.keys():
        if state in expected_states:
            expected_counts = expected_counts + counts[state]
    state_accuracy = expected_counts / shots
    return str(state_accuracy*100)+"%"

For reference, this is what the state accuracy would look like for results from a perfect backend (100%).

In [ ]:
backend = AerSimulator()
counts = backend.run(ghz, shots=shots).result().get_counts() # execution on simulator
state_accuracy = get_state_accuracy(counts)
print(f"state accuracy on the simulator: {state_accuracy}")

Note that the state accuracy is a custom metric designed for our needs. Depending on your research aim, other metrics such as the built-in [hellinger fidelity](https://qiskit.org/documentation/stubs/qiskit.quantum_info.hellinger_fidelity.html?highlight=hellinger_fidelity) might be a better choice to assess circuit execution on different devices.

Great, now we're all set to run our experiments!

## 1. Comparing IBM quantum devices<a name=devices><a/>
[[Top](#top)]

To start with, we need to find available quantum devices that are suitable to run our experiment. However, in case IBM Quantum devices are busy with jobs from other users, the execution of the following experiments take very long. To avoid this, we will use Qiskit built-in mock implementations of them, executed locally.

(If you want to try out this notebook on real systems, execute the cell above again and read the [appendix](#appendix-b) for detailed instructions.)

In [ ]:
from qiskit_ibm_runtime.fake_provider import FakeYorktownV2, FakeMelbourneV2, FakeVigoV2, FakeOurenseV2, FakeValenciaV2, FakeLondonV2, FakeBurlingtonV2, FakeEssexV2
# mock devices

backends = [FakeYorktownV2(), FakeMelbourneV2(), FakeVigoV2(), FakeOurenseV2(), FakeValenciaV2(), FakeLondonV2(),
                FakeBurlingtonV2(), FakeEssexV2()] # no mock santiago available

Now, let's compare the results from the execution of the GHZ circuit on these backends:

In [ ]:
results_backends = []

for backend in backends:
    transpiled = transpile(ghz, backend=backend, optimization_level=0)
    counts = backend.run(transpiled, shots=shots).result().get_counts() # get results
    state_accuracy = get_state_accuracy(counts) # calculate state accuracy
    results_backends.append((backend.name, counts, state_accuracy)) # store results
    
# plot results
plot_histogram([counts for backend_name, counts, state_accuracy in results_backends],
                legend=[backend_name for backend_name, counts, state_accuracy in results_backends])

The bars represent the relative amount (probability) to read a certain state from the execution of our circuit on the respective backend over the 8000 times the experiment was executed there. Remember that ideally, only four states should be returned by our circuit: 001, 010, 100, 100. Therefore, the best devices are those with the highest probability on these states while having the lowest on all other, undesired states.

To make it easier to interpret, we use the state accuracy as indicator of the results' quality:

In [ ]:
# setup ordered table

results_backends_table = pd.DataFrame(columns=["backend", "state_accuracy"]) # create table
for backend_name, counts, state_accuracy in results_backends:
    results_backends_table.loc[len(results_backends_table)] = [backend_name, state_accuracy] # put values into table
results_backends_table = results_backends_table.sort_values(by=["state_accuracy"], ascending=False).reset_index().iloc[:, 1:] # sort values
results_backends_table 


Now we know the ranking of the devices based on their fidelity. Let's pick one with a high state accuracy to test the optimization techniques in the remaining chapters. In addition, we will pick one system with a lower accuracy to compare against later.

In [ ]:
# high accuracy
backend1 = FakeVigoV2()

# lower accuracy
backend2 = FakeYorktownV2()

## 2. Manually optimizing circuits<a name=manual><a/>
[[Top](#top)]

Why is it that the IBM Quantum computer show different performances? As mentioned earlier, hardware optimization is a major research field at the moment. Essentially, our Quantum computer consist of different numbers of qubits that are connected with each other and manipulated according to our circuit. However, the way in which they are connected - the so called coupling map - as well the error rates when performing gates on single qubits, multiple qubits and when reading the state of the qubits differs across our devices due to manifacturing. Finally, the qubits posess varying coherence times (T1/T2). 

Let's have a closer look on how the coupling map and error rates look like on our two picked devices.

In [ ]:
%matplotlib inline
from qiskit.visualization import plot_gate_map
plot_gate_map(backend1)

On the Configuration tab you can see the coupling map of the devices. Qubits that are connected with a line can perform a multiple qubit gate such as CNOT directly. Trying to apply CNOT on non-connected qubits would require Qiskit to internally change the circuit before execution to fit it to the coupling map of the device. 

On the Error Map tab you can see the quality of each qubit and each connection where a darker color indicates a lower error rate (more desirable). If you are interested in the exact error rates, you can find them on the Qubit Properties tab (U2 - single qubit error rate, readout error rate) and on the Multi-Qubit Gates tab.

**Coupling Map based optimization**

The Qiskit transpiler prepares our code for execution on the real device. In case our circuit layout does not match the device's configuration, it will introduce swaps into our circuit so to make it applicable to the coupling map. Sometimes this can produce a worse accuracy than that from our original circuit. Therefore, let's try to create custom coupling map specific circuits of the GHZ state to avoid the transpiler interfering.

Backend 1:

In [ ]:
# backend1: create coupling map optimised circuit

# map: qubit 0 -> qubit 1
# qubit 1 -> qubit 0
# qubit 2 -> qubit 2

ghz_backend1 = QuantumCircuit(5,3, name=f"GHZ yyx {backend1.name} optimized") # note that we use 5 qubits here to match the configuration of the device
ghz_backend1.h(1) # we map qubit 0 to qubit 1 and qubit 1 to qubit 0 ..
ghz_backend1.cx(1,0) # .. to ensure that all logical CNOT gates are performed on physical connections
ghz_backend1.cx(1,2)
ghz_backend1.barrier()
ghz_backend1.sdg([1,0]) 
ghz_backend1.h(range(n))
ghz_backend1.barrier()
ghz_backend1.measure([1,0,2], range(n))
ghz_backend1.draw("mpl")


Let's compare the two circuits:

In [ ]:
# compare results between original and coupling map optimized circuit on backend1
results_backend1_coupling = []

circuits = [ghz, ghz_backend1]
for circuit in circuits:
    transpiled_circuit = transpile(circuit, backend=backend1, optimization_level=0)
    counts = backend1.run(transpiled_circuit, shots=shots).result().get_counts() # get results
    state_accuracy = get_state_accuracy(counts) # calculate state accuracy
    results_backend1_coupling.append((circuit.name, counts, state_accuracy)) # store results
    
# print state accuracy
print("State Accuracy")
for circuit_name, counts, state_accuracy in results_backend1_coupling:
    print(f"{circuit_name}: {state_accuracy}%")

# plot results
plot_histogram([counts for circuit_name, counts, state_accuracy in results_backend1_coupling],
                legend=[circuit_name for circuit_name, counts, state_accuracy in results_backend1_coupling])

As you can see, the state accuracy improved by around 2% on our system.

**Error Map based optimization**

As mentioned, this was one of two ways we are considering to manually optimize the circuits. The second builds off of the results from the coupling map based optimization and additionally takes into account the error rates. For this, we have another look on the error map and pick qubits that have the least single qubit gate, multiple qubit gate and readout error rate (see colors for intuition). Sometimes you need to trade off between these as for instance some qubits have excellent read out error rates while not as good single qubit gate error rates. Though, as this approach is manual, you don not need to try out each possible configuration to find the best one.

Backend 1:

In [ ]:
# backend1: create coupling map + error rate optimised circuit

# map: qubit 0 -> qubit 3
# qubit 1 -> qubit 1
# qubit 2 -> qubit 4

ghz_backend1_error = QuantumCircuit(5,3, name=f"GHZ yyx {backend1.name} error rate optimised") # note that we use 5 qubits here to match the configuration of the device
ghz_backend1_error.h(3) # qubit with best adjacent multi-qubit gates error rates (cnot)
ghz_backend1_error.cx(3,1)
ghz_backend1_error.cx(3,4) 
ghz_backend1_error.barrier()
ghz_backend1_error.sdg([3,1]) # avoid 4 as it has a worse single qubit error rate
ghz_backend1_error.h([3,1,4]) # best readout error rates
ghz_backend1_error.barrier()
ghz_backend1_error.measure([3,1,4], range(n)) # readout still in same order
ghz_backend1_error.draw("mpl")


In [ ]:
# compare results between original, coupling map, and error rates optimized circuit on backend1
results_backend1_error = []

circuits = [ghz_backend1_error]
for circuit in circuits:
    transpiled_circuit = transpile(circuit, backend=backend1, optimization_level=0)
    counts = backend1.run(transpiled_circuit, shots=shots).result().get_counts() # get results
    state_accuracy = get_state_accuracy(counts) # calculate state accuracy
    results_backend1_error.append((circuit.name, counts, state_accuracy)) # store results
    
# plot combined results
plot_histogram([counts for circuit_name, counts, state_accuracy in results_backend1_coupling + results_backend1_error],
                legend=[circuit_name for circuit_name, counts, state_accuracy in results_backend1_coupling + results_backend1_error])

As can be seen, the results from the coupling map and error rate optimised circuit are around the same as those from the circuit only optimised for its coupling map. This is because while the new circuit uses qubits with improved CNOT error rates, the old one natively had good readout error rates so that effects of swapping qubit mappings do not produce a significantly better result. However, there might be still a better circuit layout than the one found here.

## 3. Qiskit Transpiler Optimization<a name=transpiler><a/>
[[Top](#top)]

Qiskit comes with a built-in transpiler to translate logical circuits into circuits that are executable on the target backend. In addition, the transpile function (and execute function calling it) has four different levels of optimization available that can be set as parameter. See an extract of the [documentation](https://qiskit.org/documentation/apidoc/transpiler_preset.html) here to understand their differences:

Optimization Levels

* Level 0: no explicit optimization other than **mapping to backend**.

* level 1: light optimization by simple **adjacent gate collapsing**.

* level 2: medium optimization by **initial layout selection** and **gate cancellation** using commutativity rules.

* level 3: heavy optimization by **noise adaptive qubit mapping** and gate cancellation using commutativity rules and **unitary synthesis**.m

Next to other actions taken such as combining gates and mapping to the device's coupling map (which takes place necessarily on every level), note that only optimization level 3 applies noise adaptive mapping, which we attempted to do manually in the previous chapter.

To separately view the effects from each chapter, we will now again use the basic GHZ YYX circuit and compare the different transpiler optimization levels on backend 1.

In [ ]:
# compare results from different optimization levels on backend 1
results_backend1_transpiler = []

optimization_levels = [0,1,2,3]
#print(ghz)
for optim_level in optimization_levels:
    transpiled_circuit = transpile(ghz, backend=backend1, optimization_level=optim_level)
    #print(transpiled_circuit)
    counts = backend1.run(transpiled_circuit, shots=shots).result().get_counts() # get results
    state_accuracy = get_state_accuracy(counts) # calculate state accuracy
    results_backend1_transpiler.append((f"optim_level={optim_level}", counts, state_accuracy)) # store results
    
# print state accuracy
print("State Accuracy")
for circuit_name, counts, state_accuracy in results_backend1_transpiler:
    print(f"{circuit_name}: {state_accuracy}%")

# plot results
plot_histogram([counts for circuit_name, counts, state_accuracy in results_backend1_transpiler],
                legend=[circuit_name for circuit_name, counts, state_accuracy in results_backend1_transpiler])

Did you notice? Counterintuively, there was only a visible improvement from optimization level 0 to 1, and from 1 to 2, while level 2 and 3 produce similar results on the backend. But why is there no improvement on each level? To understand this phenomenon better, let's see what happens under the hood by inspecting the transpiled circuits that the transpiler produces to execute them on our backend:

In [ ]:
# compare transpiled circuits for different optimization levels on backend 1

optimization_levels = [0,1,2,3]
print("original circuit")
print(ghz)
for optim_level in optimization_levels:
    print(f"optimization level: {optim_level}")
    transpiled_circuit = transpile(ghz, backend=backend1, optimization_level=optim_level)
    print(transpiled_circuit)


The circuits of level 2 and 3 are identical! Also circuits 0 and 1 behave similarly, even though the compiler picks different qubits to map our circuit. This implies that the circuit with optimization level 2 is already the best the transpiler can do for us. The small deviation between optimization level 2 and 3 only comes due to probabilistic quantum readout randomness.

Remember that the compiler is supposed to follow the idea from our manual optimization in a more sophisticated manner. Therefore let's compare its output again with what we produced in chapter 2:

In [ ]:
print("manual optimization")
print(ghz_backend1_error)

print("State Accuracy")
print("manual optimization:  "+str(results_backend1_error[0][2]))
print("transpiler optimization: "+str(results_backend1_transpiler[3][2]))

As can be seen, the structure of the circuit remains the same even on level 3 of the transpiler optimized circuit, matching our circuit from chapter 2. However, they differ in the qubits that were selected for mapping the logical circuit onto the real physical hardware. Taking this into account, the transpiler apparently has a better intuition on how to balance the importances of different kinds of errors.

## 4. Measurement Error Mitigation<a name=mem><a/>
[[Top](#top)]

So far, we achieved some improvements of the state accuracy with manual optimization and to a small extent with transpiler optimization. While these are rather simple steps to take, there is an entire research field dealing with how to mitigate quantum computer hardware imperfection with software driven techniques, one of which we will examine now. 


As discussed earlier, there are various types of errors that can perpetrate noisy results when executing a quantum circuit. A significant one is that what happens when the quantum states are read from qubits forcing them to collapse into classical states, i.e. our experiment should return one of our four targeted classical states (001, 010, 100, or 111). However, we've seen that in our results a lot more states than these incorrectly appear. 

These returned result represent the real results plus the readout error. Hence, knowing the devices' readout error behavior, we can estimate the real results from our obtained results through matrix multiplication, which is called Measurement Error Mitigation. For this, each basis state (one of the 2^n combinations of 0 and 1 per each of the n qubits, e.g. 00, 01, 10, 11 for 2 qubits) is measured on the device so to derive its measurment errors, allowing to restore our real counts.

For a detailed explenation of how Measurement Error Mitigation works have a look at the [Qiskit Textbook](https://qiskit.org/textbook/ch-quantum-hardware/measurement-error-mitigation.html) or the well explained Qiskit [youtube video](https://www.youtube.com/watch?v=yuDxHJOKsVA) on MEM.

In [ ]:
# measurement error mitigation on backend 1
# Note: In Qiskit 2.x, measurement error mitigation is handled via Qiskit Runtime resilience levels
# For fake backends, we demonstrate the noisy execution

results_backend1_mem = []

# normal execution
transpiled = transpile(ghz, backend=backend1, optimization_level=0)
noisy_counts = backend1.run(transpiled, shots=shots).result().get_counts() # get results
noisy_state_accuracy = get_state_accuracy(noisy_counts) # calculate state accuracy from original results
results_backend1_mem.append(("noisy", noisy_counts, noisy_state_accuracy))

# Note: For real devices with Qiskit Runtime, use resilience_level parameter
# Example: sampler = Sampler(backend, options={"resilience_level": 1})

# plot results
plot_histogram([counts for name, counts, accuracy in results_backend1_mem],
                legend=[name for name, counts, accuracy in results_backend1_mem])

As can be seen, there is a visible improvement on the system with a high accuracy as result from the measurement error mitigated results.

While this is what we looked for, let's compare the results to the mitigated results from another, natively less accurate system in order to understand the capabilities of measurement error mitigation.

In [ ]:
plot_gate_map(backend2)

In [ ]:
# measurement error mitigation on backend 2

results_backend2_mem = []

# normal execution
transpiled = transpile(ghz, backend=backend2, optimization_level=0)
noisy_counts = backend2.run(transpiled, shots=shots).result().get_counts() # get results
noisy_state_accuracy = get_state_accuracy(noisy_counts) # calculate state accuracy from original results
results_backend2_mem.append(("noisy", noisy_counts, noisy_state_accuracy))

# plot results
plot_histogram([counts for name, counts, accuracy in results_backend2_mem],
                legend=[name for name, counts, accuracy in results_backend2_mem])

As can be seen, this is a truly remarkable improvement, demonstrating the power of measurement error mitigation. This implies that a large part of the erroneous results on backend 2 is due to measurement errors. In addition, it is worth noting that the effect of the measurement error mitigation fluctuates over different applications on the same backend, which in turn depends on the device's quality in the first place. Try it by executing the above cell multiple times. 

## 5. Combining the optimization techniques<a name=all><a/>
[[Top](#top)]

We've now successfully gone through a variety of optimization techniques that differ in approach, complexity and results.
However, the results for our system are still under 100%, which is what we aim for. Therefore, let's attempt to get the maximum fidelity through combining all of the measures discussed.

In [ ]:
# backend1

results_backend1_all = []

# basic circuit
transpiled = transpile(ghz, backend=backend1, optimization_level=0)
noisy_counts = backend1.run(transpiled, shots=shots).result().get_counts() # get results
noisy_state_accuracy = get_state_accuracy(noisy_counts) # calculate state accuracy from original results
results_backend1_all.append(("noisy", noisy_counts, noisy_state_accuracy))

# manually optimized circuit
transpiled = transpile(ghz_backend1_error, backend=backend1, optimization_level=0)
manual_counts = backend1.run(transpiled, shots=shots).result().get_counts() # get results
manual_state_accuracy = get_state_accuracy(manual_counts)
results_backend1_all.append(("manual", manual_counts, manual_state_accuracy))

# transpiler optimized circuit
transpiled = transpile(ghz, backend=backend1, optimization_level=3)
transpiler_counts = backend1.run(transpiled, shots=shots).result().get_counts() # get results
transpiler_state_accuracy = get_state_accuracy(transpiler_counts)
results_backend1_all.append(("transpiler", transpiler_counts, transpiler_state_accuracy))

# print state accuracy
print("State Accuracy")
for name, counts, accuracy in results_backend1_all:
    print(f"{name}: {accuracy}%")

# plot results
plot_histogram([counts for name, counts, accuracy in results_backend1_all],
                legend=[name for name, counts, accuracy in results_backend1_all])

If we wish to understand the effects better, we can inspect the transpiled circuit for each technique in order:

In [ ]:
%matplotlib inline

# inspect circuits on backend1
print("*** ",backend1.name," ***")
print("basic circuit")
print(transpile(ghz, backend=backend1, optimization_level=0))
print("manually optimized circuit (coupling map and error map)")
print(transpile(ghz_backend1_error, backend=backend1, optimization_level=0))
print("transpiler optimized circuit")
print(transpile(ghz, backend=backend1, optimization_level=3))
# (measurement error mitigation doesn't transform the circuit)

Great, we can see that the combination of methods really produces a tremendous value for the state accuracy of backend 1, close to the ideal value of 100%.

**Applying the combined techniques on another backend**

Now it's your turn! Can you combine all the methods for backend 2? To get started, we provided a possible configuration of a manually optimized circuit for backend 2:

In [ ]:
# backend 2: coupling map + error rate optimised circuit

# map: qubit 0 -> qubit 1
# qubit 1 -> qubit 2
# qubit 2 -> qubit 0

ghz_backend2_error = QuantumCircuit(5,3, name=f"GHZ yyx {backend2.name} error rate optimised") # note that we use 5 qubits here to match the configuration of the device
ghz_backend2_error.h(1) 
ghz_backend2_error.cx(1,2)
ghz_backend2_error.cx(1,0) 
ghz_backend2_error.barrier()
ghz_backend2_error.sdg([1,2]) # better single qubit error rate
ghz_backend2_error.h([1,2,0])
ghz_backend2_error.barrier()
ghz_backend2_error.measure([1,2,0], range(n)) # readout still in same order
ghz_backend2_error.draw("mpl")


Now, your task is to fill in the following cell to execute each optimization step in order just as we did with backend 1.


*(Hint: You can copy the code from the previous cell and change the variables for those relevant for backend 2. The solution can be found in the [appendix](#appendix-a) at the end of this notebook.)*

In [ ]:
### YOUR WORK ###

# backend 2

results_backend2_all = []
noisy_counts = {}
manual_counts = {}
transpiler_counts = {}
mitigated_counts = {}

# basic circuit
## YOUR CODE HERE ##
results_backend2_all.append(("noisy", noisy_counts, noisy_state_accuracy))

# manually optimized circuit
## YOUR CODE HERE ##
results_backend2_all.append(("manual", manual_counts, manual_state_accuracy))

# transpiler optimization (level 3 overrides manually optimized circuit)
## YOUR CODE HERE ##
results_backend2_all.append(("transpiler", transpiler_counts, transpiler_state_accuracy))

# transpiler optimization + measurement error mitigation
## YOUR CODE HERE ##
results_backend2_all.append(("transpiler - mitigated", mitigated_counts, mitigated_state_accuracy))

# compare state accuracies
print("State Accuracy \n"+
      "\n".join([str(label)+': '+str(state_accuracy)  for label, counts, state_accuracy in results_backend2_all]))

# plot results
plot_histogram([counts for label, counts, state_accuracy in results_backend2_all],
               legend=[label for label, counts, state_accuracy in results_backend2_all],
               title=backend2.name,
               bar_labels=False)

If you get a state accuracy of around 95% you did a good job!

Finally, we can inspect the transpiled circuits to understand the results better:

In [ ]:
# inspect circuits on backend2

print("*** ",backend2.name," ***")
print("basic circuit")
print(transpile(ghz, backend=backend2, optimization_level=0))
print("manually optimized circuit (coupling map and error map)")
print(transpile(ghz_backend2_error, backend=backend2, optimization_level=0))
print("transpiler optimized circuit")
print(transpile(ghz, backend=backend2, optimization_level=3))
# (measurement error mitigation doesn't transform the circuit)



---

Congratulations! In this tutorial, we found ways to greatly increase the probabbility with which we can win the Quantum GHZ game on a quantum computer. Please note that there is a large field of research revolving around optimising quantum hardware and software - and we only covered a few simple ones of them. Therefore feel free to dig deeper into the topic or run your own circuits on [IBM Quantum Experience](quantum-computing.ibm.com).

Thank you!

*Lennart Schulze, IBM Germany, lennart.schulze@ibm.com*

*Dr. Jan-Rainer Lahmann, IBM Germany, Jan.Lahmann@de.ibm.com*

<a name=end></a>**Important:** If running this notebook on a shared system (e.g. mybinder.org), remember to log off from your IBM Q account in case you used it for execution on real devices by running the following cell:

In [ ]:
# IBMQ account management removed - use QiskitRuntimeService instead
# See: https://docs.quantum.ibm.com/migration-guides/qiskit-runtime
pass

---
## .
## .
## .
## .


## *Appendix*<a name=appendix><a/>
[[Top](#top)]


### A: Solutions (Chapter 5)<a name=appendix-a><a/>

In [ ]:
#### SOLUTION TO CHAPTER 5 #####
 
results_backend2_all = []

# basic circuit
transpiled = transpile(ghz, backend=backend2, optimization_level=0)
noisy_counts = backend2.run(transpiled, shots=shots).result().get_counts() # get results
noisy_state_accuracy = get_state_accuracy(noisy_counts)
results_backend2_all.append(("noisy", noisy_counts, noisy_state_accuracy))

# manually optimized circuit (coupling map and error map)
transpiled = transpile(ghz_backend2_error, backend=backend2, optimization_level=0)
manual_counts = backend2.run(transpiled, shots=shots).result().get_counts()
manual_state_accuracy = get_state_accuracy(manual_counts)
results_backend2_all.append(("manual", manual_counts, manual_state_accuracy))

# transpiler optimized circuit
transpiled = transpile(ghz, backend=backend2, optimization_level=3)
transpiler_counts = backend2.run(transpiled, shots=shots).result().get_counts()
transpiler_state_accuracy = get_state_accuracy(transpiler_counts)
results_backend2_all.append(("transpiler", transpiler_counts, transpiler_state_accuracy))

# plot results
plot_histogram([counts for name, counts, accuracy in results_backend2_all],
                legend=[name for name, counts, accuracy in results_backend2_all])

---

### B: Running this notebook on real quantum computers<a name=appendix-b></a>

You can run the entire on real quantum hardware provided and made accesible by IBM on the [IBM Quantum Experience](https://quantum-computing.ibm.com).
You should take into consideration that this will take longer, as each job submitted for execution to either of the quantum computers is queued based on a fair share algorithm. Therefore, you will most likely need to wait until other jobs queued before yours are finished. You can see the status of your jobs under "Pending Results" on the [Results page](https://quantum-computing.ibm.com/results).

As the first chapter executes circuits on all suitable devices, it will take especially long. For this reason we'll provide guidance for that chapter seperately.

**Connect IBM Quantum Experience Account**

We need to connect to our IBM Quantum Experience account in order to be able to execute the results on real hardware.

If you haven't linked your IBM Quantum Experience Account yet, follow this [guide](https://qiskit.org/documentation/install.html#install-access-ibm-q-devices-label) and paste your credentials in the following cell. Remember to remove your credentials at the [end of the tutorial](#end).

In [ ]:
# For real device access, use QiskitRuntimeService:
# from qiskit_ibm_runtime import QiskitRuntimeService
# service = QiskitRuntimeService(channel="ibm_quantum", token="YOUR_TOKEN")
# service.save_account(channel="ibm_quantum", token="YOUR_TOKEN")

Having stored your account credentials, run the following to activate your account. 

In [ ]:
# Load saved account and get backend:
# from qiskit_ibm_runtime import QiskitRuntimeService
# service = QiskitRuntimeService()
# backend = service.backend("ibm_brisbane")  # or other available backend

**Using real Quantum Computers in chapter 1**

We simply need to change the "backend" variable. Just execute the following cell or use it to replace the corresponding cell above, then execute the cell in chapter 1 in which the circuits are executed and review the results.

In [ ]:
backends = provider.backends(filters=lambda x:  # get available quantum computer from IBM Q provider where
                            x.configuration().n_qubits >= n and  # number of qubits high enough
                            not x.configuration().simulator and  # only real devices (no simulator)
                            x.status().operational==True) # only devices that work
print(f"IBM Q backends: {[str(backend.name) for backend in backends]}")

**Using real Quantum Computers in chapters 2-5**

First, we need to change our backend variables for the real devices we would like to examine. You can pick the real devices corresponding to the mock ones we used above or choose your own based on the selection returned by the previous cell. (Again execute the cell here or replace the code in the corresponding cell above.)

In [ ]:
# higher accuracy
backend1 = provider.get_backend("ibmq_manila")

# lower accuracy
backend2 = provider.get_backend("ibmq_belem")

Almost done! However, as the name indicates, chapter 2 uses circuits that are manually optimized to specifically fit the default devices used so far. In order to get reasonable results in its sections, you now need to review the coupling and error map of the devices you picked by executing the first two cells in chapter 2. 

Based on this, your job is to create a new circuit for each (or decide to leave the default one if it is already fitting) that fits its coupling map, i.e. so that all qubits that interact with a cnot gate in the circuit are physically connected as indicated in the coupling map. Then, adapt the circuit again so to pick qubits with the lowest error rates. This doesn't need to be accurate, as it is hard to weigh the effects of the different kinds of errors present. A good indicator if you did a good job is that the state accuracy of the result is not significantly lower than that of the default circuit or that of the coupling map sensitive circuit without error optimization, respectively. Once you're done, execute the remaining cells in chapters 2-5.

In [ ]:
# backend1: coupling map optimised circuit

ghz_backend1 = QuantumCircuit(5,3, name=f"GHZ yyx {backend1.name} optimised") 
### YOUR CODE HERE ###


ghz_backend1.draw("mpl")

In [ ]:
# backend2: coupling map optimised circuit

ghz_backend2 = QuantumCircuit(5,3, name=f"GHZ yyx {backend2.name} optimised") 
### YOUR CODE HERE ###


ghz_backend2.draw("mpl")

In [ ]:
# backend1: coupling map + error rate optimised circuit

ghz_backend1_error = QuantumCircuit(5,3, name=f"GHZ yyx {backend1.name} error rate optimised") 
### YOUR CODE HERE ###

ghz_backend1_error.draw("mpl")

In [ ]:
# backend2: coupling map + error rate optimised circuit

ghz_backend2_error = QuantumCircuit(5,3, name=f"GHZ yyx {backend2.name} error rate optimised") 
### YOUR CODE HERE ###

ghz_backend2_error.draw("mpl")